In [ ]:
!python train.py --epochs 1 --train-photo-path train_photo_test --dataset shinkai_test --batch 4 --init-epochs 4 --use_sn 

In [ ]:
!python script/video_to_images.py --video-path ./dataset/video/city.gif

In [ ]:
!python script/edge_smooth.py --dataset shinkai --image-size 256

In [145]:
from utils.common import load_checkpoint,load_weight
from modeling.anime_ganv2 import Generator
import os
def load_checkpoint(model, checkpoint_dir, posfix=''):
    path = os.path.join(checkpoint_dir, f'{model.name}{posfix}.pth')
    return load_weight(model, path)
generator = Generator('shinkai')

start_e = load_checkpoint(G, 'checkpoint')
start_e

10

In [144]:
!python inference_image.py --checkpoint checkpoint --src dataset\train_photo\6.jpg --dest 1.png

Weight loaded, ready to predict
Anime image saved to 1.png


In [171]:
import cv2
fpath = r'dataset\train_photo\6.jpg'
image = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
image1 = np.stack([image, image, image], axis=-1)
image2 = _transform(image, addmean=False)
#image = image.transpose(2, 0, 1)

NameError: name '_transform' is not defined

In [169]:
image1.shape

(256, 256, 3)

In [170]:
image.shape

(256, 256)

In [166]:
cv2.imshow('test',image)
cv2.waitKey(0)
cv2.destroyAllWindows()